# Shortest path in a maze

This was asked during a Hackerrank.

You are given a maze, with walls, free tiles, a beginning and an end. Find the shortest path.

In [1]:
def print_mappa(mappa):
    print('#'+''.join(['####' for i in range(len(mappa[0]))])+'#')
    for row in mappa:
        print('#',end='')
        for cell in row:
            if cell == 'f' or cell == ' ' or cell == '.':
                print('   |',end='')
            elif cell == 't' or cell == '#':
                print(' # |',end='')
            elif cell == 'S':
                print(' S |',end='')
            elif cell == 'E':
                print(' E |',end='')
            elif cell == '*':
                print(' * |',end='')
            else:
                print("{:3d}|".format(cell),end='')
        print('#')
    print('#'+''.join(['####' for i in range(len(mappa[0]))])+'#')
    print()

After building a cosmetics function, give the data. The map is given as a list, to be easily modifiable. Then it is turned into a string, just to adhere to the original statement from the interview.

In [2]:
mappa = [ [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 0, 1, 1, 1],
          [0, 0, 0, 0, 0, 0, 1, 1, 0],
          [0, 1, 1, 1, 1, 1, 1, 1, 0],
          [0, 1, 1, 1, 1, 1, 1, 1, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0] ]

wall = '#'
space = '.'

for i in range(len(mappa)):
    for j in range(len(mappa[0])):
        if mappa[i][j] == 1:
            mappa[i][j] = wall
        else:
            mappa[i][j] = space

            
x_n = 0
y_n = 1
x_e = 4
y_e = 8

mappa[x_e][y_e] = 'E'
mappa[x_n][y_n] = 'S'

print_mappa(mappa)

######################################
#   | S |   |   |   |   |   |   |   |#
#   |   |   |   |   |   | # | # |   |#
# # | # | # | # | # |   | # | # |   |#
# # | # | # | # | # |   | # | # | # |#
#   |   |   |   |   |   | # | # | E |#
#   | # | # | # | # | # | # | # |   |#
#   | # | # | # | # | # | # | # |   |#
#   |   |   |   |   |   |   |   |   |#
######################################



In [3]:
mappa

[['.', 'S', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '#', '#', '.'],
 ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
 ['#', '#', '#', '#', '#', '.', '#', '#', '#'],
 ['.', '.', '.', '.', '.', '.', '#', '#', 'E'],
 ['.', '#', '#', '#', '#', '#', '#', '#', '.'],
 ['.', '#', '#', '#', '#', '#', '#', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.']]

In the following it will be so useful if a frame of walls is added around the map, helping sweeping through indices without thinking too much.

In [4]:
framed_map = []
framed_map.append([wall]+[wall for i in range(len(mappa[0]))]+[wall])
for row in mappa:
    framed_map.append([wall]+row+[wall])
framed_map.append([wall]+[wall for i in range(len(mappa[0]))]+[wall])

mappa = framed_map.copy()

rows = len(mappa)
cols = len(mappa[0])

for i in range(rows):
    for j in range(cols):
        if mappa[i][j] == 'S':
            x_n=i
            y_n=j
        if mappa[i][j] == 'E':
            x_e=i
            y_e=j

The idea is inspired by Dynamic Programming principles: assign a value to the end state, that is the finish tile, and based over this value, sweep backward the states assigning them a value depending on future states. In this case the finish tile has value 0, as the number of moves necessary to reach the end from it. The neighbouring tiles have the same value plus one, as if to say that it takes one move to reach the end tile. The neighbouring cells of the finish tile neighbours have value 2, then, and so on.

In [5]:
turno = 0
old_level = [ (x_e,y_e) ]

finito = False
while not finito:
    turno += 1
    new_level = []
    for cella in old_level:
        for (a,b) in [ (cella[0]+1,cella[1]), (cella[0]-1,cella[1]), (cella[0],cella[1]+1), (cella[0],cella[1]-1)]:
            if mappa[a][b] == space:
                mappa[a][b] = turno
                new_level.append((a,b))
            if mappa[a][b] == 'S':
                finito = True
    if len(new_level) == 0:
        raise Exception('stuck?')
    old_level = new_level

As soon as the first tile received a value, one just sweeps forward, always jumping in the tile with the lowest value. This first sweep is already the shortest path.

In [6]:
minimo = turno+1
cella = [x_n,y_n]    

finito = False
while not finito:
    for (a,b) in [ (cella[0]+1,cella[1]), (cella[0]-1,cella[1]), (cella[0],cella[1]+1), (cella[0],cella[1]-1)]:
        if type(mappa[a][b]) == int and mappa[a][b] < minimo:
            minimo = mappa[a][b]
            idx = [a,b]
        if mappa[a][b] == 'E':
            finito = True
    cella = idx
    mappa[cella[0]][cella[1]] = '*'

More cosmetics and print

In [7]:
mappa = mappa[1:-1]
for i in range(len(mappa)):
    mappa[i] = mappa[i][1:-1]

for i in range(len(mappa)):
    for j in range(len(mappa[i])):
        if mappa[i][j] not in [wall,'S','E','*']:
            mappa[i][j]='.'
        if mappa[i][j] == wall:
            mappa[i][j]='#'

for row in mappa:
    print(''.join(row))
    
print_mappa(mappa)

.S.......
.*****##.
#####*##.
#####*###
******##E
*#######*
*#######*
*********
######################################
#   | S |   |   |   |   |   |   |   |#
#   | * | * | * | * | * | # | # |   |#
# # | # | # | # | # | * | # | # |   |#
# # | # | # | # | # | * | # | # | # |#
# * | * | * | * | * | * | # | # | E |#
# * | # | # | # | # | # | # | # | * |#
# * | # | # | # | # | # | # | # | * |#
# * | * | * | * | * | * | * | * | * |#
######################################

